# Extração do Mercado Livre

## 1. Obtenção dos links dos produtos
Primeiramente, vamos extrair os links de todos os produtos.

### Instalação de bibliotecas

In [11]:
%pip install beautifulsoup4
%pip install requests
%pip install pandas
%pip install numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importando bibliotecas

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import csv

### Lista de categorias
Cada categoria tem um nome e um link, o qual será usado para obter links dos produtos em cada página.

In [254]:
category_list = [
    {
        'name': 'Celulares',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779535-1&domain_id=MLB-CELLPHONES&page='
    }, 
    {
        'name': 'Notebooks',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779536-1&domain_id=MLB-NOTEBOOKS&page='
    },
    {
        'name': 'Tênis',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779537-1&domain_id=MLB-SNEAKERS&page='
    },
    {
        'name': 'Fones',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779538-1&domain_id=MLB-HEADPHONES&page='
    },
    {
        'name': 'TVs',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779539-1&domain_id=MLB-TELEVISIONS&page='
    },
    {
        'name': 'Ferramentas',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779540-1&domain_id=MLB-WELDING_MACHINES&page='
    },
    {
        'name': 'Smartwatches',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779541-1&domain_id=MLB-SMARTWATCHES&page='
    },
    {
        'name': 'Caixas de som',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779542-1&domain_id=MLB-SPEAKERS&page='
    },
    {
        'name': 'Perfumes',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779543-1&domain_id=MLB-PERFUMES&page='
    },
    {
        'name': 'Bebês',
        'link': 'https://www.mercadolivre.com.br/ofertas?container_id=MLB773331-2&deal_ids=MLB773331-2&page='
    }
]

### Funções para requisição de páginas e extração de links

In [2]:
def request_page(url, force = True):
    success = not force
    page = None
    while not success:
        try:
            response = requests.get(url, allow_redirects=False, timeout=2)
            if response.is_redirect:
                # Ignorando este url por ter redirecionado
                break
            page = BeautifulSoup(response.text)
            success = True
        except requests.exceptions.RequestException as e:
            print(f"Erro na requisição de {url}")
            time.sleep(60)
    return page

def get_links_from_page(page):
    links = []
    if page != None: 
        link_tags = page.find_all('a',  class_ = 'promotion-item__link-container')
        for tag in link_tags:
            links.append(tag['href'])
    return links


    

### Obtendo links dos produtos de todas as categorias

In [256]:
product_links = []
# Para cada categoria
for category in category_list:
    # Gere uma lista de páginas de 1 até 20
    page_list = [f"{category['link']}{i+1}" for i in range(20)]
    category_product_total = 0
    # Para cada página desta categoria
    for page in page_list:
        page_doc = request_page(page)
        links = get_links_from_page(page_doc)
        # Já passou da última página? Pare esta categoria
        if (len(links) == 0):
            break
        # Adicione os links da página na lista de links dos produtos
        product_links.extend(links)
        category_product_total += len(links)
    print(f"Produtos da categoria {category['name']} adicionados com sucesso (Total = {category_product_total})")

Produtos da categoria Celulares adicionados com sucesso (Total = 960)
Produtos da categoria Notebooks adicionados com sucesso (Total = 780)
Produtos da categoria Tênis adicionados com sucesso (Total = 720)
Produtos da categoria Fones adicionados com sucesso (Total = 960)
Produtos da categoria TVs adicionados com sucesso (Total = 263)
Produtos da categoria Ferramentas adicionados com sucesso (Total = 960)
Produtos da categoria Smartwatches adicionados com sucesso (Total = 960)
Produtos da categoria Caixas de som adicionados com sucesso (Total = 960)
Produtos da categoria Perfumes adicionados com sucesso (Total = 960)
Produtos da categoria Bebês adicionados com sucesso (Total = 960)


In [257]:
print(f"Total de produtos: {len(product_links)}")

Total de produtos: 8483


### Salvando links dos produtos em um arquivo

In [258]:
arr = np.asarray(product_links)
np.savetxt('links.csv', arr, fmt = '%s', delimiter="\n")  

## 2. Extração dos dados dos produtos
Obtendo link do arquivo links.csv

## Funções de extração dos dados

In [3]:
# Retorna o nome do produto
def get_name(product):
    name = None
    if product is not None:
        name_tag = product.select_one('.ui-pdp-title')
        if name_tag is not None:
            name = name_tag.text
    return name

In [4]:
# Retorna o preço do produto (como string)
def get_price(product):
    price = None
    if product is not None: 
        price_tag = product.select_one('div.ui-pdp-price__second-line > .andes-money-amount.ui-pdp-price__part.andes-money-amount--cents-superscript.andes-money-amount--compact')
        if price_tag is not None:
            price_fraction = price_tag.select_one('.andes-money-amount__fraction')
            price_cents = price_tag.select_one('.andes-money-amount__cents')
            price = price_fraction.text
            if price_cents is not None:
                price += "," + price_cents.text
    return price

In [5]:
# Retorna a avaliação do produto (como string)
def get_rating(product):
    rating = None
    if product is not None:
        rating_tag = product.select('.ui-review-capability__rating__average')
        if len(rating_tag) > 0:
            rating = rating_tag[0].text
    return rating

In [6]:
# Retorna a quantidade de avaliações do produto (como string)
def get_review_count(product):
    count = None
    if product is not None:
        count_tag = product.select('.ui-pdp-review__amount')
        if len(count_tag) > 0:
            count = count_tag[0].text.replace("(", "").replace(")", "")
    return count

In [35]:
# Retorna a categoria específica do produto (como string)
def get_category(product):
    category = None
    if product is not None:
        category_tags = product.select('.andes-breadcrumb__link')
        if len(category_tags) > 0:
            category = category_tags[-1].text
    return category

In [8]:
# Retorna o termômetro do vendedor deste produto
def get_thermometer(product):
    thermometer = None
    if product is not None:
        thermometer_tag = product.select_one('.ui-thermometer')
        if thermometer_tag is not None:
            thermometer = thermometer_tag['value']
    return thermometer

In [9]:
# Retorna o nome do vendedor
def get_seller(product):
    seller = None
    if product is not None:
        seller_tag = product.select_one('.ui-pdp-color--BLUE.ui-pdp-family--REGULAR')
        if seller_tag is None:
            seller_tag = product.select_one('.ui-pdp-seller__header__title')
        if seller_tag is not None:
            seller = seller_tag.text
        if seller_tag is None:
            seller_tag = product.select_one('.ui-pdp-media__action.ui-box-component__action')
            if seller_tag is not None:
               seller = seller_tag['href'].replace('https://perfil.mercadolivre.com.br/', '')
    return seller

In [10]:
# Requisição do produto
def request_product(link, max_attempts = 1, only_if_full = False):
    result = request_page(link)
    i = 0
    while result is not None and i < max_attempts:
        i = i + 1
        if get_rating(result) is not None and get_seller(result) is not None and get_thermometer(result) is not None:
            return result
    if only_if_full:
        return None
    return result
    

In [11]:
# Para teste
link = 'https://www.mercadolivre.com.br/samsung-galaxy-a04s-dual-sim-64-gb-green-4-gb-ram/p/MLB21078576?pdp_filters=category:MLB1055%7Cdeal:MLB779535-1&hide_psmb=true#searchVariation=MLB21078576&position=1&search_layout=grid&type=product&tracking_id=597ea181-8aab-4917-a647-7be6d74825a5&deal_print_id=4b469e91-caf9-4bf7-98c5-8df304187b5a&promotion_type=DEAL_OF_THE_DAY'
link

'https://www.mercadolivre.com.br/samsung-galaxy-a04s-dual-sim-64-gb-green-4-gb-ram/p/MLB21078576?pdp_filters=category:MLB1055%7Cdeal:MLB779535-1&hide_psmb=true#searchVariation=MLB21078576&position=1&search_layout=grid&type=product&tracking_id=597ea181-8aab-4917-a647-7be6d74825a5&deal_print_id=4b469e91-caf9-4bf7-98c5-8df304187b5a&promotion_type=DEAL_OF_THE_DAY'

In [12]:
# Obtendo a página
product = request_product(link)

In [13]:
print(get_name(product))
print(get_price(product))
print(get_category(product))
print(get_rating(product))
print(get_review_count(product))
print(get_thermometer(product))
print(get_seller(product))

Samsung Galaxy A04s Dual SIM 64 GB green 4 GB RAM
839
Celulares e Smartphones
4.8
366
5
Mercado Livre


## Extraindo dados de todos os produtos
Vamos tentar extrair dados de todos os produtos e deixar o dataframe de links apenas com os produtos que falharam.

In [2]:
# Criando dataset
df_products = pd.DataFrame([], columns=['Nome', 'Preço', 'Categoria', 'Avaliação', 'Número de avaliações', 'Termômetro', 'Vendedor'])
links = []

In [3]:
# Coloca todos os dados dos produtos no data frame
# O parâmetro only_full força a retornar apenas os produtos que têm todos os campos
# Isso não vale para produtos de link https://produto.mercadolivre.com.br
def get_all_products(only_full, max_amount = 10, max_product_request = 1):
    global df_products, links
    failed_links = []
    names = []
    prices = []
    categories = []
    ratings = []
    review_counts = []
    thermometers = []
    sellers = []
    i = 0
    failed_count = 0
    for link in links:
        if i >= max_amount or (only_full and link.startswith('https://produto.mercadolivre.com.br')):
            failed_links.append(link)
            continue
        product = request_product(link, max_product_request, only_full)
        if product is None:
            failed_links.append(link) 
            failed_count += 1
            continue
        i = i + 1
        names.append(get_name(product))
        prices.append(get_price(product))
        categories.append(get_category(product))
        ratings.append(get_rating(product))
        review_counts.append(get_review_count(product))
        thermometers.append(get_thermometer(product))
        sellers.append(get_seller(product))
    
    links = failed_links
    df_added = pd.DataFrame({
        'Nome': names,
        'Preço': prices,
        'Categoria': categories,
        'Avaliação': ratings,
        'Número de avaliações': review_counts,
        'Termômetro': thermometers,
        'Vendedor': sellers
    })
    df_products = pd.concat([df_products, df_added])
    df_products = df_products.reset_index(drop=True)
    print(f"Quantidade atual de produtos: {len(df_products)}")
    print(f"Restante: {len(failed_links)}")
    print(f"Falharam: {failed_count}")
    save_dataset()
    return df_products
# Salvar dataset e lista de links restantes
def save_dataset():
    global df_products, links
    df_products.to_csv("dataset.csv", index=False)
    np.savetxt('missing_links.csv', np.asarray(links), fmt = '%s', delimiter="\n") 
# Carregar dataset e lista de links restantes
def load_dataset():
    global df_products, links
    df_products = pd.read_csv('dataset.csv')
    file = open('missing_links.csv', 'r')
    links = list(csv.reader(file, delimiter="\n"))
    links = [row[0] for row in links]
    file.close()
    return df_products

### Extraindo dados

In [30]:
# Carregando dataset obtido até o momento
load_dataset()

,Nome,Preço,Categoria,Avaliação,Número de avaliações,Termômetro,Vendedor
0,Smartphone Samsung Galaxy A14 5g 128gb 4gb Preto,1099.00,Celulares e Smartphones,4.8,205.0,5.0,Mercado Livre
1,Moto E40 Dual SIM 64 GB grafite 4 GB RAM,769.00,Celulares e Smartphones,4.6,824.0,5.0,123 Comprou
2,Samsung Galaxy S20 FE 5G 5G Dual SIM 128 GB cl...,1869.00,Celulares e Smartphones,4.8,2281.0,5.0,Mercado Livre
3,Apple iPhone 11 (128 GB) - Branco,3988.10,Celulares e Smartphones,4.5,2573.0,5.0,NILTONDASILVALIPPERTJUNIOR
4,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,1229.30,Celulares e Smartphones,4.7,4057.0,5.0,JEFERSON ELETROSTORE
...,...,...,...,...,...,...,...
8477,Paçoca Quadrada Lual Zero Açúcar Display 432g ...,30.50,Guloseimas,NaN,NaN,5.0,4WELL
8478,Óleo Essencial Natural De Cedro Virgínia 10ml Wnf,31.89,Cuidado do Corpo,NaN,NaN,5.0,LOJAS THERAPON
8479,Biscoito Maizena Vitarella Tradicional 350g - ...,23.99,Biscoitos,NaN,NaN,5.0,DISMAX+DISTRIBUIÇÃO_MÁXIMA
8480,Sabonete Íntimo Diário Instance Eudora - Fruta...,23.44,Sabonetes Íntimos,NaN,NaN,5.0,1981_VENDAS


In [312]:
# Esta célula deve ser executada várias vezes até a quantidade dos que falharam ser insignificante
# Adiciona apenas os produtos que têm todos os campos preenchidos
get_all_products(only_full=True, max_amount=1, max_product_request=1)

Quantidade atual de produtos: 1404
Restante: 7079
Falharam: 228


,Nome,Preço,Categoria,Avaliação,Número de avaliações,Termômetro,Vendedor
0,Smartphone Samsung Galaxy A14 5g 128gb 4gb Preto,1.099,Celulares e Smartphones,4.8,205,5,Mercado Livre
1,Moto E40 Dual SIM 64 GB grafite 4 GB RAM,769,Celulares e Smartphones,4.6,824,5,123 Comprou
2,Samsung Galaxy S20 FE 5G 5G Dual SIM 128 GB cl...,1.869,Celulares e Smartphones,4.8,2281,5,Mercado Livre
3,Apple iPhone 11 (128 GB) - Branco,"3.988,10",Celulares e Smartphones,4.5,2573,5,NILTONDASILVALIPPERTJUNIOR
4,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,"1.229,30",Celulares e Smartphones,4.7,4057,5,JEFERSON ELETROSTORE
...,...,...,...,...,...,...,...
1399,Truss Professional Hair Protector creme capila...,"70,13",Tratamentos com o Cabelo,4.7,62,5,MAGAZINE BRASILPREMIUM
1400,Dewar's Blended The Ancestor 12 Reino Unido 75...,"99,90",Whiskys,4.9,40,5,Mercado Livre
1401,Espumante Brasileiro Branco Brut Réserve Chand...,"59,33",Espumantes e Champagne,5.0,3,5,Empório Solera
1402,Caviar Silver 100ml,"62,90",Perfumes,3.4,5,5,Vanvic


In [46]:
# Adiciona todos os produtos, incluindo os que não têm todos os campos preenchidos
get_all_products(only_full=False, max_amount=4000)

Quantidade atual de produtos: 8482
Restante: 1
Falharam: 1


,Nome,Preço,Categoria,Avaliação,Número de avaliações,Termômetro,Vendedor
0,Smartphone Samsung Galaxy A14 5g 128gb 4gb Preto,1.099,Celulares e Smartphones,4.8,205.0,5.0,Mercado Livre
1,Moto E40 Dual SIM 64 GB grafite 4 GB RAM,769,Celulares e Smartphones,4.6,824.0,5.0,123 Comprou
2,Samsung Galaxy S20 FE 5G 5G Dual SIM 128 GB cl...,1.869,Celulares e Smartphones,4.8,2281.0,5.0,Mercado Livre
3,Apple iPhone 11 (128 GB) - Branco,"3.988,10",Celulares e Smartphones,4.5,2573.0,5.0,NILTONDASILVALIPPERTJUNIOR
4,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,"1.229,30",Celulares e Smartphones,4.7,4057.0,5.0,JEFERSON ELETROSTORE
...,...,...,...,...,...,...,...
8477,Paçoca Quadrada Lual Zero Açúcar Display 432g ...,"30,50",Guloseimas,None,None,5,4WELL
8478,Óleo Essencial Natural De Cedro Virgínia 10ml Wnf,"31,89",Cuidado do Corpo,None,None,5,LOJAS THERAPON
8479,Biscoito Maizena Vitarella Tradicional 350g - ...,"23,99",Biscoitos,None,None,5,DISMAX+DISTRIBUI%C3%87%C3%83O_M%C3%81XIMA
8480,Sabonete Íntimo Diário Instance Eudora - Fruta...,"23,44",Sabonetes Íntimos,None,None,5,1981_VENDAS


## Correção dos dados

In [5]:
# Importando bibliotecas
import urllib.parse
from math import nan

In [32]:
# Corrigindo nome do produto
def fix_name(name):
    if pd.isna(name):
        return pd.NA
    return name.strip()
# Convertendo o preço para float
def fix_price(price):
    if pd.isna(price):
        return pd.NA
    if type(price) is float:
        return price
    return float(price.replace(".", "").replace(",", "."))
# Convertendo a avaliação para float
def fix_rating(rating):
    if pd.isna(rating):
        return pd.NA
    if type(rating) is float:
        return rating
    return float(rating)
# Convertendo a avaliação para int
def fix_review_count(count):
    if pd.isna(count):
        return pd.NA
    return np.int64(count)
# Convertendo termômetro do vendedor para int
def fix_thermometer(thermometer):
    if pd.isna(thermometer):
        return pd.NA
    return np.int64(thermometer)
# Corrigindo o nome do vendedor
def fix_seller(seller):
    if seller == 'Ver mais' or pd.isna(seller):
        return pd.NA
    return urllib.parse.unquote(seller)

In [33]:
df_products['Nome'] = df_products['Nome'].apply(fix_name)
df_products['Nome']

0        Smartphone Samsung Galaxy A14 5g 128gb 4gb Preto
1                Moto E40 Dual SIM 64 GB grafite 4 GB RAM
2       Samsung Galaxy S20 FE 5G 5G Dual SIM 128 GB cl...
3                       Apple iPhone 11 (128 GB) - Branco
4       Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...
                              ...                        
8477    Paçoca Quadrada Lual Zero Açúcar Display 432g ...
8478    Óleo Essencial Natural De Cedro Virgínia 10ml Wnf
8479    Biscoito Maizena Vitarella Tradicional 350g - ...
8480    Sabonete Íntimo Diário Instance Eudora - Fruta...
8481    Chá De Maracujá - Passiflora Alata Dryander - ...
Name: Nome, Length: 8482, dtype: object

In [34]:
df_products['Preço'] = df_products['Preço'].apply(fix_price).astype('Float64').round(2)
df_products['Preço']

0       1099.0
1        769.0
2       1869.0
3       3988.1
4       1229.3
         ...  
8477      30.5
8478     31.89
8479     23.99
8480     23.44
8481      29.9
Name: Preço, Length: 8482, dtype: Float64

In [35]:
df_products['Avaliação'] = df_products['Avaliação'].apply(fix_rating).astype('Float64').round(1)
df_products['Avaliação']

0        4.8
1        4.6
2        4.8
3        4.5
4        4.7
        ... 
8477    <NA>
8478    <NA>
8479    <NA>
8480    <NA>
8481    <NA>
Name: Avaliação, Length: 8482, dtype: Float64

In [36]:
df_products['Número de avaliações'] = df_products['Número de avaliações'].apply(fix_review_count).astype('Int64')
df_products['Número de avaliações']

0        205
1        824
2       2281
3       2573
4       4057
        ... 
8477    <NA>
8478    <NA>
8479    <NA>
8480    <NA>
8481    <NA>
Name: Número de avaliações, Length: 8482, dtype: Int64

In [37]:
df_products['Termômetro'] = df_products['Termômetro'].apply(fix_thermometer).astype('Int32')
df_products['Termômetro']

0       5
1       5
2       5
3       5
4       5
       ..
8477    5
8478    5
8479    5
8480    5
8481    5
Name: Termômetro, Length: 8482, dtype: Int32

In [38]:
df_products['Vendedor'] = df_products['Vendedor'].apply(fix_seller)
df_products['Vendedor']

0                    Mercado Livre
1                      123 Comprou
2                    Mercado Livre
3       NILTONDASILVALIPPERTJUNIOR
4             JEFERSON ELETROSTORE
                   ...            
8477                         4WELL
8478                LOJAS THERAPON
8479    DISMAX+DISTRIBUIÇÃO_MÁXIMA
8480                   1981_VENDAS
8481               GRUPOROCHASAUDE
Name: Vendedor, Length: 8482, dtype: object

In [39]:
df_products

,Nome,Preço,Categoria,Avaliação,Número de avaliações,Termômetro,Vendedor
0,Smartphone Samsung Galaxy A14 5g 128gb 4gb Preto,1099.0,Celulares e Smartphones,4.8,205,5,Mercado Livre
1,Moto E40 Dual SIM 64 GB grafite 4 GB RAM,769.0,Celulares e Smartphones,4.6,824,5,123 Comprou
2,Samsung Galaxy S20 FE 5G 5G Dual SIM 128 GB cl...,1869.0,Celulares e Smartphones,4.8,2281,5,Mercado Livre
3,Apple iPhone 11 (128 GB) - Branco,3988.1,Celulares e Smartphones,4.5,2573,5,NILTONDASILVALIPPERTJUNIOR
4,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,1229.3,Celulares e Smartphones,4.7,4057,5,JEFERSON ELETROSTORE
...,...,...,...,...,...,...,...
8477,Paçoca Quadrada Lual Zero Açúcar Display 432g ...,30.5,Guloseimas,<NA>,<NA>,5,4WELL
8478,Óleo Essencial Natural De Cedro Virgínia 10ml Wnf,31.89,Cuidado do Corpo,<NA>,<NA>,5,LOJAS THERAPON
8479,Biscoito Maizena Vitarella Tradicional 350g - ...,23.99,Biscoitos,<NA>,<NA>,5,DISMAX+DISTRIBUIÇÃO_MÁXIMA
8480,Sabonete Íntimo Diário Instance Eudora - Fruta...,23.44,Sabonetes Íntimos,<NA>,<NA>,5,1981_VENDAS


In [29]:
# Salvando dataset corrigido
save_dataset()

In [40]:
# Obtendo a quantidade de células em branco para cada coluna
pd.DataFrame(df_products.isnull().sum().sort_values(), columns=["Quantidade em branco"])

,Quantidade em branco
Nome,0
Categoria,3
Preço,41
Vendedor,71
Termômetro,769
Avaliação,5058
Número de avaliações,5058


In [52]:
# Produtos que possuem todos os campos preenchidos
df_products[~df_products.isnull().any(axis=1)].reset_index(drop=True)

,Nome,Preço,Categoria,Avaliação,Número de avaliações,Termômetro,Vendedor
0,Smartphone Samsung Galaxy A14 5g 128gb 4gb Preto,1099.0,Celulares e Smartphones,4.8,205,5,Mercado Livre
1,Moto E40 Dual SIM 64 GB grafite 4 GB RAM,769.0,Celulares e Smartphones,4.6,824,5,123 Comprou
2,Samsung Galaxy S20 FE 5G 5G Dual SIM 128 GB cl...,1869.0,Celulares e Smartphones,4.8,2281,5,Mercado Livre
3,Apple iPhone 11 (128 GB) - Branco,3988.1,Celulares e Smartphones,4.5,2573,5,NILTONDASILVALIPPERTJUNIOR
4,Xiaomi Redmi Note 11 (Snapdragon) Dual SIM 128...,1229.3,Celulares e Smartphones,4.7,4057,5,JEFERSON ELETROSTORE
...,...,...,...,...,...,...,...
3290,Loção Hidrata & Revitaliza Para Corpo Neutroge...,28.41,Cuidado do Corpo,4.7,3,5,Mercado Livre Beauty
3291,Erva-mate Tertúlia chimarrão alto vácuo 1 kg,27.2,Erva-Mate,5.0,3,5,LOJAO-DO-TERERE
3292,Colgate Luminous White Lovers Café Branqueador...,13.56,Creme Dental,5.0,2,5,Mercado Livre
3293,Kit Babosa No Cabelo Forever Liss Com Shampoo ...,26.99,Tratamentos com o Cabelo,4.0,2,5,Forever Liss
